In [ ]:
import os
os.chdir( "../")

In [ ]:
import pandas as pd
import numpy as np

path = "Dataset/data_train.csv"
df = pd.read_csv(filepath_or_buffer=path,
                               sep=",",
                               header=1,
                               engine='python',
                               names=['UserID', 'ItemID', 'Interaction'])


df

In [ ]:
df.Interaction.value_counts()

In [ ]:
df.info()

In [ ]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

In [ ]:
df.head()

In [ ]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample



urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation, urm_test = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80)
urm_train, urm_validation = split_train_in_two_percentage_global_sample(urm_train_validation, train_percentage = 0.80)

In [ ]:
num_users = len(userId_unique)
num_items = len(itemId_unique)

In [ ]:
from Recommenders.Recommender_import_list import *
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10], ignore_users=[])

## Insert model here

In [ ]:
from Recommenders.EASE_R.EASE_R_RecommenderV2 import EASE_R_RecommenderV2
model = EASE_R_RecommenderV2

## Hyperparamters Tuning

In [ ]:
import optuna as op

def objective(trial):


    hyperparameters_range_dictionary = {
                "epochs": trial.suggest_categorical("epochs",[500]),
                "learning_rate": trial.suggest_float("learning_rate",low=1e-6, high=1e-2, log=True),
                "l2_reg": trial.suggest_float("l2_reg",low=1e-6, high=1e-2, log=True),
                "dropout": trial.suggest_float("dropout", low=0., high=0.8),
                "total_anneal_steps": trial.suggest_int("total_anneal_steps",100000, 600000),
                "anneal_cap": trial.suggest_float("anneal_cap",low=0., high=0.6),
                "batch_size": trial.suggest_categorical("batch_size",[128, 256, 512, 1024]),
                "p_dims": trial.suggest_categorical("p_dims",[[num_users, num_items]]),
                "sgd_mode": trial.suggest_categorical("sgd_mode",["Adam"]),
            }




     # Earlystopping hyperparameters available in the framework
    full_hyperp = {"validation_every_n": 3,
                   "stop_on_validation": True,
                   "evaluator_object": evaluator_validation,
                   "lower_validations_allowed": 3,   # Higher values will result in a more "patient" earlystopping
                   "validation_metric": "MAP",
                  }
                      

    
    recommender = model(urm_train, verbose=True)
    recommender.fit(**trial.params, **full_hyperp)
    
    # Add the number of epochs selected by earlystopping as a "user attribute" of the optuna trial
    epochs = recommender.get_early_stopping_final_epochs_dict()["epochs"]
    trial.set_user_attr("epochs", epochs) 

    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
study = op.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

In [ ]:
study.best_value

In [ ]:
study.best_params

## Fit and Test

In [ ]:
final = model(urm_train_validation)
final.fit(**study.best_params)

In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)